In [1]:
import numpy as np 
import pandas as pd 
import transformers 

import torch
from torch.utils.data import DataLoader

In [2]:
df = pd.read_csv('/kaggle/input/llm-sum/dssum1.csv')

In [3]:
df.tail(10)

,Unnamed: 0,a,b,target
629991,9990,8.073030e+18,5.075405e+18,1.314844e+19
629992,9991,6.582864e+18,8.505424e+18,1.508829e+19
629993,9992,6.422259e+18,5.115904e+18,1.153816e+19
629994,9993,5.863110e+18,4.613692e+18,1.047680e+19
629995,9994,5.250378e+18,8.996710e+18,1.424709e+19
629996,9995,7.610591e+18,5.220341e+18,1.283093e+19
629997,9996,4.831392e+18,8.249815e+18,1.308121e+19
629998,9997,6.888173e+18,6.751073e+18,1.363925e+19
629999,9998,5.095527e+18,8.920606e+18,1.401613e+19
630000,9999,6.282287e+18,8.368257e+18,1.465054e+19


In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [24]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large')

In [25]:
model = model.to(device)

In [72]:
text = "god is "
encoded_input = tokenizer(text, return_tensors='pt').to(device)
output = model(**encoded_input)

In [73]:
output.logits.argmax(-1)

tensor([[  11,  262, 1849]], device='cuda:0')

In [74]:
tokenizer.decode(output.logits.argmax(-1)[0])

', the\xa0'

In [5]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2-large', device=device)

In [10]:
import dataclasses

@dataclasses.dataclass
class Example:
    question: str
    answer: str
    thought: str 

In [14]:
SUM_EXAMPLES = [
    Example(
        question='159 + 453',
        answer='612',
        thought= 'Start calculating from the tail of question 9 + 3 = 12, then 2 is a last digit of answer and 1 is an additional number for the next position, then 5 + 5 = 10, also add additional number 1, 10 + 1 = 11, then 1 is the second digit from the tail and 1 is an additional digit for next position, next pair to sum 1 + 4 = 5, also add an additional number 1, 5 + 1 = 6, then the answer is 612'
    ),
    Example(
        question='125 + 489',
        answer='614',
        thought='Start calculation from the tail of question 9 + 5 = 14, then 4 is a last digit of answer and 1 is an additional number for the next position, then 2 + 8 = 10, also add addition number 1, 10 + 1 = 11, then 1 is the second digit from the tail and 1 is an additional digit for the next position, next pair to sum is 1 + 4 = 5, also add an additional number 1, 5 + 1= 6, then the naswer is 614'
    )
]

In [147]:
train = df.sample(2000)

In [20]:
BATCH_SIZE = 40 

In [149]:
from tqdm.notebook import tqdm 

In [155]:
init_prompt = ''
for ex in SUM_EXAMPLES:
    init_prompt += 'Q: '   + ex.question + '\nA: ' + ex.thought + ' The answer is ' + ex.answer  + '.\n\n'

In [159]:
output = []
for ix in tqdm(range(0, train.shape[0], BATCH_SIZE)):
    input_list = []
    for ix, row in df.iloc[ix:ix+BATCH_SIZE].iterrows():
        prompt = init_prompt + f'Q: {row.a} + {row.b}\nThe answer is:'
        input_list.append(prompt)
    output.append(generator(input_list,
                   max_length=253, num_return_sequences=5))

  0%|          | 0/50 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [162]:
import re 

pattern = r"The answer is: (\d+)"
def extract_answers(out):
    res = re.search(pattern, out['generated_text'], re.MULTILINE)
    if not res:
        return None 
    return int(res.group(1))

In [163]:
preds = list(map(lambda y: list(map(lambda x: list(map(extract_answers, x)), y)), output))

In [166]:
final_preds = sum(preds, [])

In [167]:
def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num

In [170]:
final_preds = list(map(most_frequent, final_preds))

In [172]:
from sklearn.metrics import accuracy_score

In [180]:
accuracy_score(y_true=train['target'].astype(int).astype('str'), y_pred =list(map(str, final_preds)))

0.0

In [179]:
train['target'].astype(int).astype('str')

63794                     4167
29081                     9953
625074    -9223372036854775808
15005                    14242
127683                   29137
                  ...         
198688                 1650648
198056                 1653100
122425                   19472
220697                11233026
546295       55876915320626568
Name: target, Length: 2000, dtype: object